In [29]:
import requests
from bs4 import BeautifulSoup
import re
%load_ext autoreload
import pandas as pd


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:

# What does our URL look like?
url = "https://www.bbcgoodfood.com/recipes/feta-peach-couscous"
# Get the response
response = requests.get(url)
# Turn it into Soup
soup = BeautifulSoup(response.text, "html.parser")
# Find the right tag

prep_time = soup.find("div", class_= "icon-with-text__children").find_all('time')
times=[prep_time[0].text.strip(' mins'), prep_time[1].text.strip(' mins')]
print(times)

['5', '15']


In [15]:


# URL of the search results page
search_url = "https://www.bbcgoodfood.com/search?q="

# Send an HTTP GET request
response = requests.get(search_url)
soup = BeautifulSoup(response.content, "html.parser")

# Extract recipe links
recipe_links = []
for link in soup.find_all("a", class_="link d-block"):
    recipe_links.append(f'https://www.bbcgoodfood.com/recipes{link["href"]}')

In [16]:
recipe_links



['https://www.bbcgoodfood.com/recipes/recipes/lemon-drizzle-cake',
 'https://www.bbcgoodfood.com/recipes/recipes/chicken-chorizo-jambalaya',
 'https://www.bbcgoodfood.com/recipes/recipes/best-ever-chocolate-brownies-recipe',
 'https://www.bbcgoodfood.com/recipes/recipes/creamy-courgette-lasagne',
 'https://www.bbcgoodfood.com/recipes/recipes/chilli-con-carne-recipe',
 'https://www.bbcgoodfood.com/recipes/recipes/naughty-chocolate-fudge-cake',
 'https://www.bbcgoodfood.com/recipes/recipes/classic-victoria-sandwich-recipe',
 'https://www.bbcgoodfood.com/recipes/recipes/chicken-noodle-soup',
 'https://www.bbcgoodfood.com/recipes/recipes/satay-sweet-potato-curry',
 'https://www.bbcgoodfood.com/recipes/recipes/chicken-pasta-bake',
 'https://www.bbcgoodfood.com/recipes/recipes/best-spaghetti-bolognese-recipe',
 'https://www.bbcgoodfood.com/recipes/recipes/brilliant-banana-loaf',
 'https://www.bbcgoodfood.com/recipes/recipes/chorizo-mozzarella-gnocchi-bake',
 'https://www.bbcgoodfood.com/reci

In [17]:
cuisines = ['italian','indian','asian']
dietary= ['vegetarian','vegan', 'gluten-free','nut-free']

base_url= 'https://www.bbcgoodfood.com/search?tab=recipe'

def preference_based_search(preference=None):
    if preference in cuisines:
        url= f'{base_url}&cuisines={preference}'
    elif preference in dietary:
        url= f'{base_url}&diet={preference}'
    else:
        url=base_url
    return url

In [7]:
print(preference_based_search('italian'))

https://www.bbcgoodfood.com/search?tab=recipe&cuisines=italian


In [21]:
def category_specific_links(preference):
    search_url = preference_based_search(preference)
    response = requests.get(search_url)
    soup = BeautifulSoup(response.content, "html.parser")
    recipe_links = []
    for link in soup.find_all("a", class_="link d-block"):
        recipe_links.append(f'https://www.bbcgoodfood.com/recipes{link["href"]}')
    return recipe_links

In [9]:
import pandas as pd

list1 = [1,2,3,4,5]
list2 = [4,5,6,7,8]
dat={'d':list1, 'f':list2}

df1= pd.DataFrame(dat)

list3 = [3,42,53,4,5]
list4 = [4,55,66,7,8]
datr={'d':list3, 'f':list4}



In [18]:
# %reload_ext autoreload
from replenish.get_data.bbc_scrape import load_data



In [20]:
df = load_data()
df

In [26]:
category_specific_links('italian')

['https://www.bbcgoodfood.com/recipes/recipes/lemon-drizzle-cake',
 'https://www.bbcgoodfood.com/recipes/recipes/chicken-chorizo-jambalaya',
 'https://www.bbcgoodfood.com/recipes/recipes/best-ever-chocolate-brownies-recipe',
 'https://www.bbcgoodfood.com/recipes/recipes/creamy-courgette-lasagne',
 'https://www.bbcgoodfood.com/recipes/recipes/chilli-con-carne-recipe',
 'https://www.bbcgoodfood.com/recipes/recipes/naughty-chocolate-fudge-cake',
 'https://www.bbcgoodfood.com/recipes/recipes/classic-victoria-sandwich-recipe',
 'https://www.bbcgoodfood.com/recipes/recipes/chicken-noodle-soup',
 'https://www.bbcgoodfood.com/recipes/recipes/satay-sweet-potato-curry',
 'https://www.bbcgoodfood.com/recipes/recipes/chicken-pasta-bake',
 'https://www.bbcgoodfood.com/recipes/recipes/best-spaghetti-bolognese-recipe',
 'https://www.bbcgoodfood.com/recipes/recipes/brilliant-banana-loaf',
 'https://www.bbcgoodfood.com/recipes/recipes/chorizo-mozzarella-gnocchi-bake',
 'https://www.bbcgoodfood.com/reci

In [39]:
def category_bbc_data(preference):
    '''Return a dataframe of recipe information for each recipe
    given a specific category'''

    individual_recipes = category_specific_links(preference)

    #list for each item retrieved
    recipe_title=[]
    prep_times =[]
    cooking_times=[]
    stars=[]
    review_count=[]
    difficulty_level=[]
    servings=[]
    description=[]
    recipe_ingredients=[]

    #here the individual recipe links are pulled and iterated
    for recipe in individual_recipes:
        #time.sleep(1)
        response = requests.get(recipe)
        soup = BeautifulSoup(response.content, "html.parser")

        #recipe title
        try:
            title = soup.find("div", class_= "headline post-header__title post-header__title--masthead-layout").find(
            "h1",class_='heading-1').string
            recipe_title.append(title)
        except:
            recipe_title.append('n')

        #prep and cooking times
        try:
            times = soup.find("div", class_= "icon-with-text__children").find_all('time')
            prep_times.append(times[0].text.strip(' mins'))
            cooking_times.append(times[1].text.strip(' mins'))
        except:
            prep_times.append('n')
            cooking_times.append('n')

        try:
        #star ratings
            star_rating = soup.find("div", class_= "rating__values").find("span",class_='sr-only').string
            star_rating_float = float(star_rating.strip('A star rating of ').strip(' out of 5.'))
            stars.append(star_rating_float)
        except:
            stars.append('n')

        try:
        #review count
            num_reviews = soup.find("div", class_= "rating__values").find("span",class_='rating__count-text body-copy-small').string
            num_reviews_float = float(num_reviews.strip(' ratings'))
            review_count.append(num_reviews_float)
        except:
            review_count.append('n')

        #servings
        #if soup.find("div", class_= "icon-with-text post-header__servings body-copy-small body-copy-bold icon-with-text--aligned") not None:
        try:
            serving = soup.find("div", class_= "icon-with-text post-header__servings body-copy-small body-copy-bold icon-with-text--aligned").find("div", class_= "icon-with-text__children").string
        #servings_int = int(serving.strip('Serves '))
            servings.append(serving)
        except:
            servings.append("None")

        try:
        #description
            tagline = soup.find("div", class_= "editor-content mt-sm pr-xxs hidden-print").find("p").string
            description.append(tagline)
        except:
            description.append('n')

        try:

            #difficulty level
            difficulty = soup.find("div", class_= "icon-with-text post-header__skill-level body-copy-small body-copy-bold icon-with-text--aligned").find("div", class_= "icon-with-text__children").string
            difficulty_level.append(difficulty)
        except:
            difficulty_level.append('n')


        try:
    
            #ingredients
            for ingredient_group in soup.find_all('section', class_='recipe__ingredients col-12 mt-md col-lg-6'):
                ingredients = ingredient_group.find_all('li')
                ingredient_text = ''
            for ingredient in ingredients:
                ingredient_text += ingredient.get_text() + ', '
            recipe_ingredients.append(ingredient_text)
        except:
            recipe_ingredients.append('n')

        #Creating a Dictionary
    category_dictionary= {
        'recipe_title':recipe_title,
        'stars':stars,
        'prep_times':prep_times,
        'cooking_times': cooking_times,
        'review_count':review_count,
        'difficulty_level':difficulty_level,
        'servings':servings,
        'description':description,
        'ingredients':recipe_ingredients
    }

    #Dictionary to dataframe
    df = pd.DataFrame.from_dict(category_dictionary)

    return df

In [40]:
category_bbc_data('italian')

,recipe_title,stars,prep_times,cooking_times,review_count,difficulty_level,servings,description,ingredients
0,Chicken & chorizo jambalaya,4.8,10,45,2560.0,Easy,Serves 4,A healthy Cajun-inspired rice pot recipe that'...,"1 tbsp olive oil, 2 chicken breasts, chopped, ..."
1,Lemon drizzle cake,4.7,15,45,2765.0,Easy,Cuts into 10 slices,It's difficult not to demolish this classic le...,"225g unsalted butter, softened, 225g caster s..."
2,Best ever chocolate brownies recipe,4.8,25,27,2550.0,More effort,Cuts into 16 squares or 32 triangles,A super easy brownie recipe for a squidgy choc...,"185g unsalted butter, 185g best dark chocolate..."
3,Chilli con carne recipe,4.8,10,1 hr,2516.0,Easy,Serves 4,This great chilli recipe has to be one of the ...,"1 large onion, 1 red pepper, 2 garlic cloves, ..."
4,Creamy courgette lasagne,4.6,10,20,643.0,Easy,Serves 4,"Serve this quick, creamy courgette & ricotta l...","9 dried lasagne sheets, 1 tbsp sunflower oil,..."
5,Classic Victoria sandwich recipe,4.0,40,20,1062.0,Easy,Cuts into 10 slices,"The perfect party cake, a Victoria sponge is a...","200g caster sugar, 200g softened butter, 4 egg..."
6,Easy chocolate fudge cake,4.7,25,30,985.0,Easy,Serves 8,Need a guaranteed crowd-pleasing cake that's e...,"150ml sunflower oil, plus extra for the tin, 1..."
7,Chicken noodle soup,4.8,10,30,752.0,Easy,Serves 2,Mary Cadogan's aromatic broth will warm you up...,900ml chicken or vegetable stock (or Miso soup...
8,Sweet potato & peanut curry,4.7,15,45,940.0,Easy,Serves 4,"Cook this tasty, budget-friendly vegan curry f...","1 tbsp coconut oil, 1 onion, chopped, 2 garlic..."
9,Chicken pasta bake,4.6,30,45,605.0,Easy,Serves 6,Enjoy this gooey cheese and chicken pasta bake...,"4 tbsp olive oil, 1 onion, finely chopped, 2 g..."


In [36]:
print(len(recipe_title))
print(len(prep_times ))
print(len(cooking_times))
print(len(stars))
print(len(review_count))
print(len(difficulty_level))
print(len(servings))
print(len(description))
print(len(recipe_ingredients))

33
30
33
33
33
33
33
33
33


In [28]:
url = 'https://www.bbcgoodfood.com/recipes/crispy-asian-salmon-stir-fried-noodles-pak-choi-sugar-snap-peas'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

In [29]:

ingredient_list = []
for ingredient_group in soup.find_all('section', class_='recipe__ingredients col-12 mt-md col-lg-6'):
    ingredients = ingredient_group.find_all('li')
    ingredient_text = ''
    for ingredient in ingredients:
        ingredient_text += ingredient.get_text(strip=True) + ', '
    ingredient_list.append(ingredient_text)
print(ingredient_list)

["2 x 100gsalmon fillets(plus 2 more 100g salmon fillets if cooking for Flaked salmon salad lunch - see 'goes well with'), 2 tspreduced salt tamari or soy sauce, 2cm pieceginger,peeled and finely chopped or grated, 1garlic clove,finely chopped, 2 tbsplemon or lime juice, 1 tspsesame oil, 85gvermicelli rice noodle, 2 tsprapeseed oil, 1 tspsesame oil, 1spring onion,trimmed and thinly sliced, 1garlic clove,finely chopped, ½red chilli,deseeded and finely chopped, 2cm pieceginger,peeled and finely chopped, 100gsugar snap pea, 100gpak choi(or spinach), 1large red pepper,sliced, 1 tsptamari or soy sauce, 1 tspThai fish sauce, juice ½lime, 1 tbspfinely chopped coriander, "]
